In [ ]:
import os
import tempfile
import shutil
from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, avg, max as spark_max, min as spark_min, stddev, sum as spark_sum, when, desc
from pyspark.conf import SparkConf
from pyspark import SparkContext
from minio import Minio
import warnings
warnings.filterwarnings("ignore")

In [ ]:
MINIO_ENDPOINT = os.environ.get("MINIO_ENDPOINT", "localhost:9000")
MINIO_ACCESS_KEY = os.environ.get("MINIO_ACCESS_KEY", "minioadmin")
MINIO_SECRET_KEY = os.environ.get("MINIO_SECRET_KEY", "minioadmin")

minio_client = Minio(MINIO_ENDPOINT, access_key=MINIO_ACCESS_KEY, secret_key=MINIO_SECRET_KEY, secure=False)
print("✅ MinIO conectado")

In [ ]:
try:
    sc = SparkContext.getOrCreate()
    sc.stop()
except:
    pass
import time
time.sleep(1)
conf = SparkConf().setAppName("GenerarKPIs").setMaster("local[*]").set("spark.driver.bindAddress", "127.0.0.1").set("spark.driver.host", "127.0.0.1")
try:
    sc = SparkContext(conf=conf)
    spark = SparkSession(sc)
except:
    spark = SparkSession.builder.config(conf=conf).getOrCreate()
print("✅ Spark iniciado")

In [ ]:
archivo_silver = None
try:
    print("📥 Buscando archivos Silver...")
    objects = minio_client.list_objects("meteo-silver", recursive=True)
    archivos_silver = [obj.object_name for obj in objects if obj.object_name.endswith(".csv")]
    if archivos_silver:
        archivo_silver = sorted(archivos_silver)[-1]
        print(f"✅ Cargando: {archivo_silver}")
        temp_dir = tempfile.gettempdir()
        temp_file = os.path.join(temp_dir, archivo_silver.split("/")[-1])
        minio_client.fget_object("meteo-silver", archivo_silver, temp_file)
        df = spark.read.csv(temp_file, header=True, inferSchema=True)
        print(f"✅ Cargados {df.count()} registros")
    else:
        print("⚠️ Sin archivos Silver")
        df = spark.createDataFrame([(1, 25.5, 60, "2024-01-01")], ["sensor_id", "temperature", "humidity", "timestamp"])
except Exception as e:
    print(f"⚠️ Error: {e}")
    df = spark.createDataFrame([(1, 25.5, 60, "2024-01-01")], ["sensor_id", "temperature", "humidity", "timestamp"])
print(f"📊 DataFrame: {df.count()} registros")
df.show(3)

In [ ]:
print("\n📊 MÉTRICAS GENERALES:")
metricas_generales = df.agg(
    count("*").alias("total_registros"),
    avg("temperature").alias("temp_promedio"),
    spark_max("temperature").alias("temp_maxima"),
    spark_min("temperature").alias("temp_minima"),
    stddev("temperature").alias("temp_desviacion"),
    avg("humidity").alias("humedad_promedio"),
    spark_max("humidity").alias("humedad_maxima"),
    spark_min("humidity").alias("humedad_minima")
).collect()[0]

for key, value in metricas_generales.asDict().items():
    print(f"  {key}: {value}")

In [ ]:
print("\n📊 MÉTRICAS POR SENSOR:")
metricas_sensor = df.groupBy("sensor_id").agg(
    count("*").alias("cantidad_lecturas"),
    avg("temperature").alias("temp_promedio"),
    spark_max("temperature").alias("temp_max"),
    spark_min("temperature").alias("temp_min"),
    avg("humidity").alias("humedad_promedio")
).orderBy(desc("cantidad_lecturas"))
metricas_sensor.show()

In [ ]:
print("\n📊 MÉTRICAS DE CALIDAD:")
metricas_calidad = df.agg(
    count(when(col("temperature").isNotNull(), 1)).alias("registros_temp_validos"),
    count(when(col("humidity").isNotNull(), 1)).alias("registros_humedad_validos"),
    count(when((col("temperature") >= 0) & (col("temperature") <= 50), 1)).alias("temp_en_rango"),
    count(when((col("humidity") >= 0) & (col("humidity") <= 100), 1)).alias("humedad_en_rango")
).collect()[0]

for key, value in metricas_calidad.asDict().items():
    print(f"  {key}: {value}")

In [ ]:
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
archivo_gold = f"metricas_kpi_gold_{timestamp}.csv"
print(f"💾 Generando: {archivo_gold}")

try:
    kpi_df = df.groupBy("sensor_id").agg(
        count("*").alias("lecturas"),
        avg("temperature").alias("temp_avg"),
        spark_max("temperature").alias("temp_max"),
        spark_min("temperature").alias("temp_min"),
        stddev("temperature").alias("temp_std"),
        avg("humidity").alias("hum_avg"),
        spark_max("humidity").alias("hum_max"),
        spark_min("humidity").alias("hum_min")
    )
    
    temp_file_local = os.path.join(tempfile.gettempdir(), archivo_gold)
    with open(temp_file_local, "w") as f:
        rows = kpi_df.collect()
        cols = kpi_df.columns
        f.write(",".join(cols) + "\\n")
        for row in rows:
            f.write(",".join([str(row[c]) if row[c] is not None else "" for c in cols]) + "\\n")
    
    minio_client.fput_object("meteo-gold", archivo_gold, temp_file_local)
    print(f"✅ {archivo_gold} guardado en Gold")
    os.remove(temp_file_local)
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback; traceback.print_exc()

In [ ]:
print("\n" + "="*60)
print("✅ GENERACIÓN DE KPIs COMPLETADA")
print(f"Origen: {archivo_silver}")
print(f"Destino: meteo-gold/{archivo_gold}")
print("="*60)